# Tutorial 3: Custom Runtime

GenC can be customized for your own product domain. In this tutorial we explore how the following topics:

* `CustomFunction` - define arbirary function and make it an operator in GenC, for instance, a parsers of model output can be modeled as custom function.
* Domain Specific Tools as `Intrinsics` - define own GenAI buidling blocks, for instance wolfram is a tool.
* `ModelInference` - is a GenC predefined intrinsics that allows user defined model calls. It's helpful for you to standardize multiple model backends and easily switch between them.
* `ExecutorStack` which will be able to keep global context and bring all building blocks together under one Custom runtime for your use cases.

To motivate this reading, in the next tutorial, we'll use all these building blocks to create LLM Agents.

note: if you are interested in GenC internal after this tutorial, read more from [extensibility API documentation](api.md)

## 1. CustomFunction

These are user defined functions that can be mix-n-matched with native GenC
building blocks.

For example, models often takes a structured (such as JSON) input and output.
However, in a typical "chain", we want the operator that calls the model to be
text-in-text-out. This means, there're two helper components in the model call
chain:

*   Format model input - given text, format it into a JSON request for the model
*   Parse model output - give a JSON response, parse it into a text

These are inline functions can be executed by itself. Therefore we model it as
`CustomFunction`. Let's take a look how this is done.

### 1.1 Define the function interface

These are plain C++ functions you would write without GenC. There are two steps it would take to make it work with GenC.

1. The function signiture is Value-in-Value-out. GenC `Value` is a proto that can carry multimodal data. This would make the function chainable to other operator regardless of your data type.
   
   * `static absl::StatusOr<v0::Value> GetTopCandidateAsText(v0::Value input);`
   * `static absl::StatusOr<v0::Value> WrapTextAsInputJson(v0::Value input);`

2. We need to expose these functions to the runtime.
  `static absl::Status SetCustomFunctions(intrinsics::CustomFunction::FunctionMap& fn_map);` allows that. see [extensibility API documentation](api.md) to understand where it comes from.


Under [`gemini_parser.h`](../cc/modules/parsers/gemini_parser.h)
```c++
// Parsers for Gemini model.
class GeminiParser final {
 public:
  ~GeminiParser() = default;

  // Extract Top Candidate as Text.
  static absl::StatusOr<v0::Value> GetTopCandidateAsText(v0::Value input);

  // Wraps a text as Gemini request JSON.
  static absl::StatusOr<v0::Value> WrapTextAsInputJson(v0::Value input);

  // Make Parser functions visible to the runtime.
  static absl::Status SetCustomFunctions(
      intrinsics::CustomFunction::FunctionMap& fn_map);

  // Not copyable or movable.
  GeminiParser(const GeminiParser&) = delete;
  GeminiParser& operator=(const GeminiParser&) = delete;

 private:
  // Do not hold states in this class.
  GeminiParser() = default;
};
```

### 1.3 Define the functions.

Here we have an input formatter to Gemini and an output parser.

Under [`gemini_parser.cc`](../cc/modules/parsers/gemini_parser.cc)
``` c++
absl::StatusOr<v0::Value> GeminiParser::GetTopCandidateAsText(v0::Value input) {
  auto parsed_json = nlohmann::json::parse(input.str(), /*cb=*/nullptr,
                                           /*allow_exceptions=*/false);
  if (parsed_json.is_discarded()) {
    return absl::InternalError(absl::Substitute(
        "Failed parsing json output from Gemini: $0", input.DebugString()));
  }

  std::string extract_first_candidate_as_text =
      "{% if candidates %}{% for p in candidates.0.content.parts "
      "%}{{p.text}}{% endfor %}{%   endif %}";

  inja_status_or::Environment env;

  v0::Value result;
  std::string result_str =
      GENC_TRY(env.render(extract_first_candidate_as_text, parsed_json));
  result.set_str(result_str);
  return result;
}

absl::StatusOr<v0::Value> GeminiParser::WrapTextAsInputJson(v0::Value input) {
  std::string json_request = absl::Substitute(
      R"pb(
        {
          "contents":
          [ {
            "parts":
            [ { "text": "$0" }]
          }]
        }
      )pb",
      input.str());
  v0::Value result;
  result.set_str(json_request);
  return result;
}
```

### 1.4 Make your functions visible to the runtime

These are simple Lamda that wraps the newly defined functions. Notice that the routing uri such as `/gemini_parser/get_top_candidate_as_text` is totally up to your choice, it simply tells the runtime which function to invoke when it see an incoming compuation.

Under `gemini_parser.cc`
```c++
absl::Status GeminiParser::SetCustomFunctions(
    intrinsics::CustomFunction::FunctionMap& fn_map) {
  fn_map["/gemini_parser/get_top_candidate_as_text"] =
      [](const v0::Value& arg) {
        return GeminiParser::GetTopCandidateAsText(arg);
      };

  fn_map["/gemini_parser/wrap_text_as_input_json"] = [](const v0::Value& arg) {
    return GeminiParser::WrapTextAsInputJson(arg);
  };

  return absl::OkStatus();
}
```

### 1.5 Wire it into the runtime

Under `executor_stack.cc`

```c++
GENC_TRY(GeminiParser::SetCustomFunctions(config.custom_function_map));
```

A word on executor stack:

* `config` is a global context, you define it in your executor_stack.
* the executor stacks we provide is for your conveninece, in practice you can create your own. The benefit is that each stack contians only the modules you need and nothing more.

If you are interested read futher in Section 4: Executor Stacks.
Also see [extensibility API documentation](api.md)


### 1.6 Try it out.

Combine rest call and I/O parsers to form a custom model call chain.
These are available in C++ also with similar syntax, please check our c++ examples.

A python example can be located in [math_tool_agent.py](../python/examples/math_tool_agent.py)
A c++ example can be located in [run_gemini_on_ai_studio.cc](../cc/examples/run_gemini_on_ai_studio.cc)




In [ ]:
import generative_computing as genc

rest_call = genc.authoring.create_rest_call("<end point with api key>")
str_to_json_request = genc.authoring.create_custom_function(
    "/gemini_parser/wrap_text_as_input_json"
)
extrat_top_candidate = genc.authoring.create_custom_function(
    "/gemini_parser/get_top_candidate_as_text"
)

model_call = (
    genc.interop.langchain.CustomChain()
    | str_to_json_request
    | rest_call
    | extrat_top_candidate
)

model_call_chain = genc.interop.langchain.create_computation(model_call)
runner = genc.runtime.Runner(model_call_chain)
runner("Tell me a story")

## Intrinsics
You can consider them as native building blocks of your custom SDK.




## ModelInference
GenC provides some predefined ways to call popular model backends such as ChatGPT, Gemini. The reality is that you'll wan to customize them for your own use cases.

There are many model backends out there. Gemini, ChatGPT, self hosted LLAMA... each comes with a different input and output. Is there a way to bring all of them under a consistent interface, so you can easily chain them with other operators or switch between diffent models? This section provides the solution to this question.

## ExecutorStack

A custom runtime that holds context/state and brings all building blocks together.

## Intro to next tutorial: Buidling Modular Agents

Congratualtions you just made enough modular components to build a more interesting LLM agents. Next tutorial we'll see in action how all these building blocks will come together to create an LLM agent.